In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

In [2]:
people = pd.read_csv('Traffic_Crashes_-_People.csv')
crashes = pd.read_csv('Traffic_Crashes_-_Crashes.csv')

people['CRASH_YEAR'] = people['CRASH_DATE'].apply(lambda x : int(x.split()[0].split('/')[-1])) 

crashes['CRASH_YEAR'] = crashes['CRASH_DATE'].apply(lambda x : int(x.split()[0].split('/')[-1])) 

crashes = crashes[crashes['CRASH_YEAR']==2021]

people = people[people['CRASH_YEAR']==2021]

C:\Users\Sajal\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3186: DtypeWarning: Columns (20,24,25,26,29) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## Deleting Columns with mostly missing data

In [3]:
def find_sparse_columns(df, threshold):
    ans = []
    for col in df.columns:
        if df[col].isna().sum() > threshold:
            ans.append(col)
    return ans

In [4]:
find_sparse_columns(people, 100000)

['SEAT_NO',
 'DRIVERS_LICENSE_STATE',
 'DRIVERS_LICENSE_CLASS',
 'HOSPITAL',
 'EMS_AGENCY',
 'EMS_RUN_NO',
 'PEDPEDAL_ACTION',
 'PEDPEDAL_VISIBILITY',
 'PEDPEDAL_LOCATION',
 'BAC_RESULT VALUE',
 'CELL_PHONE_USE']

In [5]:
people.drop(find_sparse_columns(people, 100000), axis=1, inplace=True)

In [6]:
crashes.shape

(108755, 50)

In [7]:
find_sparse_columns(crashes, 104000)

['LANE_CNT',
 'PHOTOS_TAKEN_I',
 'STATEMENTS_TAKEN_I',
 'DOORING_I',
 'WORK_ZONE_I',
 'WORK_ZONE_TYPE',
 'WORKERS_PRESENT_I']

In [8]:
crashes.isna().sum()

CRASH_RECORD_ID                       0
RD_NO                                 0
CRASH_DATE_EST_I                 100054
CRASH_DATE                            0
POSTED_SPEED_LIMIT                    0
TRAFFIC_CONTROL_DEVICE                0
DEVICE_CONDITION                      0
WEATHER_CONDITION                     0
LIGHTING_CONDITION                    0
FIRST_CRASH_TYPE                      0
TRAFFICWAY_TYPE                       0
LANE_CNT                         108739
ALIGNMENT                             0
ROADWAY_SURFACE_COND                  0
ROAD_DEFECT                           0
REPORT_TYPE                        3375
CRASH_TYPE                            0
INTERSECTION_RELATED_I            83057
NOT_RIGHT_OF_WAY_I               103533
HIT_AND_RUN_I                     69928
DAMAGE                                0
DATE_POLICE_NOTIFIED                  0
PRIM_CONTRIBUTORY_CAUSE               0
SEC_CONTRIBUTORY_CAUSE                0
STREET_NO                             0


In [9]:
crashes.drop(find_sparse_columns(crashes, 100000), axis=1, inplace=True)

In [10]:
crashes.shape

(108755, 41)

In [11]:
people.dropna(axis=0, inplace=True)
crashes.dropna(axis=0, inplace=True)

## Function that will add a new column - time elaspsed since start of day

In [12]:
def time_calc(s):
    ans = 0
    if s.split()[-1]=='AM':
        if int(s.split()[1].split(':')[0]) != 12:
            ans += 60*int(s.split()[1].split(':')[0]) + int(s.split()[1].split(':')[1])
        else:
            ans += int(s.split()[1].split(':')[1])
    else:
        ans += 12*60
        if int(s.split()[1].split(':')[0]) == 12:
            ans += int(s.split()[1].split(':')[1])
        else:
            ans += 60*int(s.split()[1].split(':')[0]) + int(s.split()[1].split(':')[1])
    return ans

## Merging the datasets

In [16]:
merge1 = pd.merge(people, crashes, on='CRASH_RECORD_ID', how='inner')

merge1.shape

merged_sample = merge1

merged_sample.shape

merge1['TIME'] = merge1['CRASH_DATE_x'].apply(time_calc)

merge1['TIME'].describe()
merge1 = merge1.sample(2500)

merge1.to_csv('Traffic_Crashes_Medium2.csv', index=False)